In [ ]:
semillas <- c(677213, 727817, 311237, 660719, 106427)

In [ ]:
# Instalar paquetes faltantes
install.packages("rlist")
install.packages("DiceKriging")
install.packages("mlrMBO")

# Cargar paquetes
library(data.table)
library(rlist)
library(rpart)
library(parallel)
library(DiceKriging)
library(mlrMBO)


In [3]:
#Optimizacion Bayesiana de hiperparametros de  rpart

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("rlist")

require("rpart")
require("parallel")

#paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")


#Defino la  Optimizacion Bayesiana

kBO_iter  <- 100   #cantidad de iteraciones de la Optimizacion Bayesiana

hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 4000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )             # minbuket NO PUEDE ser mayor que la mitad de minsplit

ksemilla_azar  <- 677213   #cambiar por la primer semilla

#------------------------------------------------------------------------------
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0( folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}
#------------------------------------------------------------------------------
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 
# particionar( data=dataset, division=c(1,1,1,1,1), agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}
#------------------------------------------------------------------------------
#fold_test  tiene el numero de fold que voy a usar para testear, entreno en el resto de los folds
#param tiene los hiperparametros del arbol

ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],  #entreno en todo MENOS el fold_test que uso para testing
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo, 
                          data[ fold==fold_test, ],  #aplico el modelo sobre los datos de testing
                          type= "prob")   #quiero que me devuelva probabilidades

  prob_baja2  <- prediccion[, "BAJA+2"]  #esta es la probabilidad de baja

  #calculo la ganancia
  ganancia_testing  <- data[ fold==fold_test ][ prob_baja2 > 1/40,  
                                                sum( ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ) )] 

  return( ganancia_testing )  #esta es la ganancia sobre el fold de testing, NO esta normalizada
}
#------------------------------------------------------------------------------

ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos

  particionar( data, divi, seed=semilla, agrupa=pagrupa )  #particiono en dataset en folds

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )   #se puede subir a qfolds si posee Linux o Mac OS

  data[ , fold := NULL ]

  #devuelvo la primer ganancia y el promedio
  ganancia_promedio  <- mean( unlist( ganancias ) )   #promedio las ganancias
  ganancia_promedio_normalizada  <- ganancia_promedio * qfolds  #aqui normalizo la ganancia
  
  return( ganancia_promedio_normalizada )
}
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <- ArbolesCrossValidation( dataset,
                                        param= x, #los hiperparametros del arbol
                                        qfolds= xval_folds,  #la cantidad de folds
                                        pagrupa= "clase_ternaria",
                                        semilla= ksemilla_azar )

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   xx$iteracion <- GLOBAL_iteracion
   loguear( xx,  arch= archivo_log )

   return( ganancia )
}
#------------------------------------------------------------------------------
#Aqui empieza el programa

setwd(".")   #Establezco el Working Directory

#cargo los datos
dataset  <- fread("../datasets/dataset_pequeno.csv")
#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]


#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/HT3210/", showWarnings = FALSE )
setwd("./exp/HT3210/")   #Establezco el Working Directory DEL EXPERIMENTO


archivo_log  <- "HT321.txt"
archivo_BO   <- "HT321.RDATA"

#leo si ya existe el log, para retomar en caso que se se corte el programa
GLOBAL_iteracion  <- 0

if( file.exists(archivo_log) )
{
 tabla_log  <- fread( archivo_log )
 GLOBAL_iteracion  <- nrow( tabla_log )
}



#Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar  <- EstimarGanancia

configureMlr( show.learner.output= FALSE)

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= archivo_BO)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <- makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

#inicio la optimizacion bayesiana
if( !file.exists( archivo_BO ) ) {

  run  <- mbo( fun=     obj.fun, 
               learner= surr.km,
               control= ctrl)

} else  run  <- mboContinue( archivo_BO )   #retomo en caso que ya exista



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2149995,114.9,4250637,227.1,2975991,159
Vcells,3741275,28.6,8388608,64.0,8388607,64


Computing y column(s) for design. Not provided.



20230430 135357	-0.317384413564287	5740	2849	14	5	49956000	1
20230430 135418	-0.682395569476648	5053	2424	9	5	50391000	2
20230430 135434	-0.458169154466304	4999	1290	5	5	51231000	3
20230430 135503	-0.81662905812409	2050	56	12	5	54078000	4
20230430 135524	-0.0170150642457882	7404	3304	11	5	49557000	5
20230430 135536	0.0942951177203213	6301	2244	19	5	0	6
20230430 135557	-0.961971155609353	3732	1753	8	5	52605000	7
20230430 135625	-0.874425867183891	6649	265	15	5	49350000	8
20230430 135641	-0.225711673694604	7704	568	6	5	50589000	9
20230430 135706	-0.704165950244559	3746	550	10	5	52911000	10
20230430 135727	-0.676042545982636	6822	2679	16	5	50565000	11
20230430 135750	-0.215972195282978	5422	1621	11	5	50706000	12
20230430 135814	-0.159611638868227	7819	1746	15	5	50589000	13
20230430 135833	-0.218332983305057	7195	1412	8	5	50589000	14
20230430 135854	-0.718950437987223	7448	3663	20	5	50196000	15
20230430 135910	0.0107251847162844	2702	823	6	5	0	16


[mbo] 0: cp=-0.317; minsplit=5740; minbucket=2849; maxdepth=14 : y = 5e+07 : 22.8 secs : initdesign

[mbo] 0: cp=-0.682; minsplit=5053; minbucket=2424; maxdepth=9 : y = 5.04e+07 : 21.2 secs : initdesign

[mbo] 0: cp=-0.458; minsplit=4999; minbucket=1290; maxdepth=5 : y = 5.12e+07 : 15.6 secs : initdesign

[mbo] 0: cp=-0.817; minsplit=2050; minbucket=56; maxdepth=12 : y = 5.41e+07 : 29.0 secs : initdesign

[mbo] 0: cp=-0.017; minsplit=7404; minbucket=3304; maxdepth=11 : y = 4.96e+07 : 21.0 secs : initdesign

[mbo] 0: cp=0.0943; minsplit=6301; minbucket=2244; maxdepth=19 : y = 0 : 12.2 secs : initdesign

[mbo] 0: cp=-0.962; minsplit=3732; minbucket=1753; maxdepth=8 : y = 5.26e+07 : 20.8 secs : initdesign

[mbo] 0: cp=-0.874; minsplit=6649; minbucket=265; maxdepth=15 : y = 4.94e+07 : 28.1 secs : initdesign

[mbo] 0: cp=-0.226; minsplit=7704; minbucket=568; maxdepth=6 : y = 5.06e+07 : 16.3 secs : initdesign

[mbo] 0: cp=-0.704; minsplit=3746; minbucket=550; maxdepth=10 : y = 5.29e+07 : 24.

20230430 135922	-0.0831679976360584	7999	3985	3	5	49557000	17


...

[mbo] 98: cp=-0.382; minsplit=2855; minbucket=168; maxdepth=14 : y = 5.19e+07 : 30.5 secs : infill_ei



20230430 144020	-0.824423717555402	7995	3997	14	5	49557000	115


[mbo] 99: cp=-0.824; minsplit=7995; minbucket=3997; maxdepth=14 : y = 4.96e+07 : 20.4 secs : infill_ei



20230430 144040	-0.668786765169118	1202	471	6	5	54504000	116


[mbo] 100: cp=-0.669; minsplit=1202; minbucket=471; maxdepth=6 : y = 5.45e+07 : 18.4 secs : infill_ei

Saved the final state in the file HT321.RDATA

